In [1]:
import requests
import pandas as pd
import numpy as np
import time
from bs4 import BeautifulSoup
import random
import os
import datetime

In [2]:
stock ={"陽明":"2609","長榮":"2603","萬海":"2615","長榮航":"2618","遠雄港":"5607","華航":"2610","台驊投控":"2636","裕民":"2606",
        "益航":"2601" ,"東森":"2614","新興":"2605" , "四維航":"5608" , "台航":"2617" }
url_list = [f"https://goodinfo.tw/StockInfo/ShowK_ChartFlow.asp?RPT_CAT=PER&STOCK_ID={i}&CHT_CAT=MONTH" for i in stock.values()]

In [3]:
day_now =datetime.datetime.strftime(datetime.datetime.today() ,'%Y%m%d') #現在的月份
date=pd.date_range(start="20170101",end=day_now ,freq="MS" ).map(lambda x : x.strftime("%Y%m"))

In [4]:
headers ={"user-agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"}

In [5]:
def get_request(url , headers):
    #request
    r = requests.get(url ,headers=headers)
    r.encoding="utf-8"
    res = r.text
    soup=BeautifulSoup( res ,"lxml")
    data = soup.select_one("#divDetail")
    return data
    


In [6]:
#需要的最前的時間
want_time = "17M01" 
def get_read_html(data  , want_time):
    #bs4 解析
    
    dfs=pd.read_html(data.prettify())[0]#prettify() : 函數將我們的data物件美化所用
    dfs.columns = dfs.iloc[0,:]
    #去除不需要的文字欄位
    index_drop =[]
    for i , j in zip(dfs.iloc[:,4] ,dfs.iloc[:,4].index.values):
        if "EPS" in i :
            index_drop.append(j)
    df_new=dfs.drop(index=index_drop).reset_index().drop(columns="index")
    #找到時間的起點，依照該表單月份的格式去做到搜尋
    want_index=df_new[df_new.iloc[:,0]==f"{want_time}"].index.values[0]+1
    df_new=df_new.iloc[:want_index ,:]
    return df_new  #回傳一個整理好的data



In [7]:
# 決定要儲存的資料夾
output_dir = 'Data_EPS和PER' 

# 假如資料夾不存在就新增一個資料夾
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [8]:
for name , code  , url in zip(stock ,stock.values() , url_list ):
    print(url)
    data=get_request(url , headers)
    time.sleep(15)#停每15秒，避免阻擋爬蟲
    while data  is None:
        data=request_And_bs4(url , headers)
        time.sleep(15)#停每15秒，避免阻擋爬蟲
    df_new = get_read_html(data , want_time)
    df=pd.read_csv(f"./Data_price/{name}_{code}.csv" ,index_col=["Unnamed: 0"] )#讀取資料
    df["EPS" ]= df_new.iloc[:,4].map(lambda x : float(x))[::-1].values #轉型成float，倒轉以符合時間
    df["PER_medain_price"] = (df["medain_price"]/df["EPS" ]).map(lambda x : 0 if x<=0 or x==float("inf") or x==float("-inf") else x).round(2)
    df.iloc[:,4:].to_csv(f"./Data_EPS和PER/{name}_{code}.csv")


https://goodinfo.tw/StockInfo/ShowK_ChartFlow.asp?RPT_CAT=PER&STOCK_ID=2609&CHT_CAT=MONTH
https://goodinfo.tw/StockInfo/ShowK_ChartFlow.asp?RPT_CAT=PER&STOCK_ID=2603&CHT_CAT=MONTH
https://goodinfo.tw/StockInfo/ShowK_ChartFlow.asp?RPT_CAT=PER&STOCK_ID=2615&CHT_CAT=MONTH
https://goodinfo.tw/StockInfo/ShowK_ChartFlow.asp?RPT_CAT=PER&STOCK_ID=2618&CHT_CAT=MONTH
https://goodinfo.tw/StockInfo/ShowK_ChartFlow.asp?RPT_CAT=PER&STOCK_ID=5607&CHT_CAT=MONTH
https://goodinfo.tw/StockInfo/ShowK_ChartFlow.asp?RPT_CAT=PER&STOCK_ID=2610&CHT_CAT=MONTH
https://goodinfo.tw/StockInfo/ShowK_ChartFlow.asp?RPT_CAT=PER&STOCK_ID=2636&CHT_CAT=MONTH
https://goodinfo.tw/StockInfo/ShowK_ChartFlow.asp?RPT_CAT=PER&STOCK_ID=2606&CHT_CAT=MONTH
https://goodinfo.tw/StockInfo/ShowK_ChartFlow.asp?RPT_CAT=PER&STOCK_ID=2601&CHT_CAT=MONTH
https://goodinfo.tw/StockInfo/ShowK_ChartFlow.asp?RPT_CAT=PER&STOCK_ID=2614&CHT_CAT=MONTH
https://goodinfo.tw/StockInfo/ShowK_ChartFlow.asp?RPT_CAT=PER&STOCK_ID=2605&CHT_CAT=MONTH
https://go